<a href="https://colab.research.google.com/github/salisuyahaya/Cotton-Disease-Prediction-Deep-Learning/blob/master/Hausa_Sentiment_analysis(Algorithms).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!unzip "/content/hausa-sentiment-analysis.zip"

In [141]:
!pip install catboost


     |████████████████████████████████| 76.1 MB 65 kB/s 


In [142]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import re
import string
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import xgboost as xgb 
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import GridSearchCV


from sklearn.svm import SVC

from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics
#import optuna
#from optuna.trial import TrialState

from sklearn import svm

from catboost import CatBoostClassifier



from numpy import array



**LOAD DATA**

In [4]:
train = pd.read_csv('/content/train (2).csv')
test = pd.read_csv('/content/test (1).csv')

**EDA + VISUALIZATIONS**

In [5]:
#Get summary of the training data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        240 non-null    int64 
 1   NewsText  240 non-null    object
 2   label     240 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.8+ KB


In [6]:
#display first rows of the training dataset
train.head()

,id,NewsText,label
0,1553,Sannan ya ringa yin fina-finai a kan matsalol...,1
1,1059,Karanta cikakken labarin a nan,0
2,1006,Tuni dai Saudiyya ta ce za ta kaddamar da aiki...,1
3,1242,Wannan zai hana raguwar darajarsa kamar yadda ...,-1
4,1221,"Ba shi da wata iyaka, ma'ana iya shafukan da z...",0


In [ ]:
pos = []
neg = []
neut = []

for label in train.label:
    if label == 0:
        pos.append(0)
        neg.append(0)
        neut.append(1)
    elif label == 1:
        pos.append(1)
        neg.append(0)
        neut.append(0)
    elif label == -1:
          pos.append(0)
          neg.append(1)
          neut.append(0)

In [ ]:
train['Pos']= pos
train['Neg']= neg
train['Neut']= neut

###Data Cleaning(Train)

In [7]:
train.head()

,id,NewsText,label
0,1553,Sannan ya ringa yin fina-finai a kan matsalol...,1
1,1059,Karanta cikakken labarin a nan,0
2,1006,Tuni dai Saudiyya ta ce za ta kaddamar da aiki...,1
3,1242,Wannan zai hana raguwar darajarsa kamar yadda ...,-1
4,1221,"Ba shi da wata iyaka, ma'ana iya shafukan da z...",0


In [8]:
def remove_punct(text1):
    text_nopunct1 = ''
    text_nopunct1 = re.sub('['+string.punctuation+']', '', text1)
    return text_nopunct1                                                                        
                                                                                                  
train['Text_Clean1'] = train['NewsText'].apply(lambda x: remove_punct(x))

In [9]:
import nltk                                                                                    
nltk.download('punkt')
                                                                                                  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
from nltk import word_tokenize, WordNetLemmatizer
tokens1 = [word_tokenize(sen) for sen in train.Text_Clean1]                                                                 

In [11]:
def lower_token1(tokens1): 
    return [w.lower() for w in tokens1]                                                         
                                                                                      
lower_tokens1 = [lower_token1(token) for token in tokens1] 

In [12]:
import nltk
nltk.download('stopwords')                                                                         


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
from nltk.corpus import stopwords
stoplist1 = stopwords.words('english')                                                       

In [14]:
def remove_stop_words(tokens1):                                                               
    return [word for word in tokens1 if word not in stoplist1]                              

In [15]:
filtered_words1 = [remove_stop_words(sen1) for sen1 in lower_tokens1]                          

In [16]:
result1 = [' '.join(sen1) for sen1 in filtered_words1]                                                  

In [17]:
train['Text_Final1'] = result1                                                                                

In [18]:
train['tokens1'] = filtered_words1                                                                                 

In [19]:
train = train[['Text_Final1', 'tokens1', 'label']]         

In [20]:
train[:4]                                                                                       


,Text_Final1,tokens1,label
0,sannan ya ringa yin finafinai kan matsalolin z...,"[sannan, ya, ringa, yin, finafinai, kan, matsa...",1
1,karanta cikakken labarin nan,"[karanta, cikakken, labarin, nan]",0
2,tuni dai saudiyya ta ce za ta kaddamar da aiki...,"[tuni, dai, saudiyya, ta, ce, za, ta, kaddamar...",1
3,wannan zai hana raguwar darajarsa kamar yadda ...,"[wannan, zai, hana, raguwar, darajarsa, kamar,...",-1


In [21]:
all_train_words = [word for tokens1 in train["tokens1"] for word in tokens1]
train_sentence_lengths = [len(tokens1) for tokens1 in train["tokens1"]]
TRAIN_VOCAB1 = sorted(list(set(all_train_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_train_words), len(TRAIN_VOCAB1)))
print("Max sentence length is %s" % max(train_sentence_lengths))

4490 words total, with a vocabulary size of 1412
Max sentence length is 46


###Data Cleaning(Test)

In [22]:
test.head()

,id,NewsText
0,1278,"Mene ne sakamakon zaben?""Hukumar zabe ta ayyan..."
1,1721,Gwamnatin Najeriya ta ce an sako yaran ne lami...
2,1180,"""An biya kuɗin fansa kafin a sake su, amma ba ..."
3,1009,Rahotanni sun ce an shafe tsawon sa’a takwas a...
4,1588,Ya karbi lambobin yabo a gida Najeriya da kuma...


In [23]:
def remove_punct(text2):
    text_nopunct2 = ''
    text_nopunct2 = re.sub('['+string.punctuation+']', '', text2)
    return text_nopunct2                                                                         
                                                                                                  
test['Text_Clean2'] = test['NewsText'].apply(lambda x: remove_punct(x))

In [24]:
import nltk                                                                                    
nltk.download('punkt')
                                                                                                  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
from nltk import word_tokenize, WordNetLemmatizer
tokens2 = [word_tokenize(sen2) for sen2 in test.Text_Clean2]                                                                  

In [26]:
def lower_token2(tokens2): 
    return [w.lower() for w in tokens2]                                                         
                                                                                      
lower_tokens2 = [lower_token2(token) for token in tokens2] 

In [27]:
import nltk
nltk.download('stopwords')                                                                              


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
from nltk.corpus import stopwords
stoplist2 = stopwords.words('english')                                                       

In [29]:
def remove_stop_words(tokens2):                                                              
    return [word for word in tokens2 if word not in stoplist2]                              

In [30]:
filtered_words2 = [remove_stop_words(sen2) for sen2 in lower_tokens2]                          

In [31]:
result2 = [' '.join(sen2) for sen2 in filtered_words2]                                                 

In [32]:
test['Text_Final2'] = result2                                                                              

In [33]:
test['tokens2'] = filtered_words2          
                                                                         

In [34]:
test['tokens2'] = filtered_words2                                                                                   

In [35]:
test = test[['Text_Final2', 'tokens2']]                   

In [36]:
test[:4]                                                                                       

,Text_Final2,tokens2
0,mene ne sakamakon zabenhukumar zabe ta ayyana ...,"[mene, ne, sakamakon, zabenhukumar, zabe, ta, ..."
1,gwamnatin najeriya ta ce sako yaran ne lami la...,"[gwamnatin, najeriya, ta, ce, sako, yaran, ne,..."
2,biya kuɗin fansa kafin sake su amma ba zan fad...,"[biya, kuɗin, fansa, kafin, sake, su, amma, ba..."
3,rahotanni sun ce shafe tsawon sa ’ takwas ana ...,"[rahotanni, sun, ce, shafe, tsawon, sa, ’, tak..."


In [37]:
#from pandas import DataFrame

In [38]:
#df1 = pd.DataFrame.from_records([tokes2])

In [39]:
test['liststring2'] = test['tokens2'].agg(lambda x: ','.join(map(str, x)))


In [40]:
test.head()

,Text_Final2,tokens2,liststring2
0,mene ne sakamakon zabenhukumar zabe ta ayyana ...,"[mene, ne, sakamakon, zabenhukumar, zabe, ta, ...","mene,ne,sakamakon,zabenhukumar,zabe,ta,ayyana,..."
1,gwamnatin najeriya ta ce sako yaran ne lami la...,"[gwamnatin, najeriya, ta, ce, sako, yaran, ne,...","gwamnatin,najeriya,ta,ce,sako,yaran,ne,lami,la..."
2,biya kuɗin fansa kafin sake su amma ba zan fad...,"[biya, kuɗin, fansa, kafin, sake, su, amma, ba...","biya,kuɗin,fansa,kafin,sake,su,amma,ba,zan,fad..."
3,rahotanni sun ce shafe tsawon sa ’ takwas ana ...,"[rahotanni, sun, ce, shafe, tsawon, sa, ’, tak...","rahotanni,sun,ce,shafe,tsawon,sa,’,takwas,ana,..."
4,ya karbi lambobin yabo gida najeriya da kuma waje,"[ya, karbi, lambobin, yabo, gida, najeriya, da...","ya,karbi,lambobin,yabo,gida,najeriya,da,kuma,waje"


In [41]:
train['liststring1'] = train['tokens1'].agg(lambda x: ','.join(map(str, x)))


In [42]:
train.head()

,Text_Final1,tokens1,label,liststring1
0,sannan ya ringa yin finafinai kan matsalolin z...,"[sannan, ya, ringa, yin, finafinai, kan, matsa...",1,"sannan,ya,ringa,yin,finafinai,kan,matsalolin,z..."
1,karanta cikakken labarin nan,"[karanta, cikakken, labarin, nan]",0,"karanta,cikakken,labarin,nan"
2,tuni dai saudiyya ta ce za ta kaddamar da aiki...,"[tuni, dai, saudiyya, ta, ce, za, ta, kaddamar...",1,"tuni,dai,saudiyya,ta,ce,za,ta,kaddamar,da,aiki..."
3,wannan zai hana raguwar darajarsa kamar yadda ...,"[wannan, zai, hana, raguwar, darajarsa, kamar,...",-1,"wannan,zai,hana,raguwar,darajarsa,kamar,yadda,..."
4,ba shi da wata iyaka maana iya shafukan da za ...,"[ba, shi, da, wata, iyaka, maana, iya, shafuka...",0,"ba,shi,da,wata,iyaka,maana,iya,shafukan,da,za,..."


# MODELS

**Logistic Regresson**

In [43]:
#vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['liststring1'])
test_matrix = vectorizer.transform(test['liststring2'])

In [44]:
X_train1 = train_matrix
X_test1 = test_matrix


In [143]:
grid = {'max_depth': [3,4,5],'n_estimators':[100, 200, 300]}

#param_grid = {'n_estimators': [100, 150, 200, 250, 300,], 'learning_rate': [0.0001, 0.001, 0.01, 0.1]}

In [147]:
#The model
gs_ab = GridSearchCV(estimator = cbc, param_grid = grid, scoring ='accuracy', cv = 5)


#rdfs = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)(C=1e5)
#rdfs = RandomForestClassifier(max_depth=100, max_features='sqrt', min_samples_leaf=2, 
#min_samples_split=10, n_estimators=800) 

#svc=SVC(probability=True, kernel='linear')


#gs_ab = GridSearchCV(AdaBoostClassifier(), param_grid=param_grid)

#abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)

#abc = AdaBoostClassifier(XGBClassifier())
#abc = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200)
#abc = AdaBoostClassifier(RandomForestClassifier(max_depth=100, max_features='sqrt', min_samples_leaf=2, min_samples_split=10, n_estimators=800)
#gbm = XGBClassifier(learning_rate=0.3, Random_state=5,n_estimator=200)

#dst = tree.DecisionTreeClassifier(max_depth=3)

#nb=MultinomialNB()

#clf = svm.SVC(C=100, class_weight='balanced', gamma=0.5, kernel='linear', probability=True) 

#lr = LogisticRegression(C=0.1)
#lr = LogisticRegression(random_state=42)
#regressor = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
 #             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
  #            importance_type='gain', interaction_constraints='',
              #learning_rate=0.300000012, max_delta_step=0, max_depth=6,
#           min_child_weight=1, missing=0, monotone_constraints=1,
#          n_estimators=100, n_jobs=12, num_parallel
####      validate_parameters=1, verbosity=3))




NameError: ignored

In [123]:
#lr = LogisticRegression(C=0.1)
#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
   #             max_depth = 5, alpha = 10, n_estimators = 10)



In [124]:
#gb=GradientBoostingClassifier(learning_rate=0.01, n_estimators=1500,max_depth=4, min_samples_split=40, min_samples_leaf=7,max_features=4 , subsample=0.95, random_state=10)


In [125]:
#from sklearn.model_selection import GridSearchCV
#lr= LogisticRegression()
#clf_CV= GridSearchCV(lr, param_grid= 0.1)
#clf_CV.fit(X_train1,train['label'])
gs_ab.fit(X_train1,train['label'])




GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1],
                         'n_estimators': [100, 150, 200, 250, 300, 400, 500]})

In [126]:
#Make predictions

predictions1=gs_ab.predict(X_test1)
len(predictions1)
#predictions1=abc.predict(X_test1)
#len(predictions1)
#predictions1=regressor.predict(X_test1)
#len(predictions1)


60

In [127]:
print(predictions1[:10])

[ 1  1  1 -1  1  1 -1  0  1  0]


Submission File

In [128]:
sample_submission = pd.read_csv('/content/sample_submission (2).csv')

In [129]:
sample_submission.head()

,id,label
0,1278,NaN
1,1721,NaN
2,1180,NaN
3,1009,NaN
4,1588,NaN


In [130]:
submission = {'id':sample_submission['id'],
              'label':predictions1}

In [131]:
submission_file = pd.DataFrame(submission)
submission_file.head()

,id,label
0,1278,1
1,1721,1
2,1180,1
3,1009,-1
4,1588,1


In [132]:
submission_file.shape

(60, 2)

In [133]:
submission_file.set_index('id', inplace=True)
submission_file.head()

,label
id,
1278,1
1721,1
1180,1
1009,-1
1588,1


In [134]:
submission_file.to_csv('submission_file.csv')

In [135]:
submission_file-submission_file.head()

,label
id,
1005,NaN
1007,NaN
1009,0.0
1017,NaN
1024,NaN
1025,NaN
1033,NaN
1042,NaN
1045,NaN
